# NA 568 Mobile Robotics: Methods \& Algorithms Winter 2021 -- Homework 2 -- Kalman Filtering

University of Michigan\
January 25, 2021

## Task 5. Estimation (40 points)
Assume that we want to estimate an unobserved population parameter $\theta$ on the basis of observations $x$. Let $f$ be the sampling distribution of $x$ so that $f(x|\theta)$ is the probability of $x$ when the underlying population parameter is $\theta$. The function $L(\theta) = f(x|\theta)$ when viewed as a function of the parameter $\theta$ is called the likelihood function or just the likelihood.
For example, if $x$ follows a Gaussian distribution, we will have $\theta = (\mu, \sigma^2)$ and $$\mu, \sigma \mapsto f(x|\mu,\sigma^2) = \frac{1}{\sigma \sqrt{2\pi}}\exp(-\frac{1}{2}\frac{(x-\mu)^2}{\sigma^2})$$ is the likelihood. 

**Maximum Likelihood Estimator (MLE):** The maximum likelihood method maximizes the likelihood function, leading to the MLE
\begin{equation} 
    \nonumber \hat{\theta}_{MLE} = \text{argmax}_\theta L(\theta) = \text{argmax}_\theta f(x|\theta).
\end{equation}

**Maximum A Posteriori (MAP) Estimator:** In the Bayesian framework, one can place a prior distribution over the parameter, i.e., $g(\theta)$. Then, the MAP estimator maximizes the posterior probability density function $f(\theta|x)$ as follows.
\begin{equation}\nonumber
    \hat{\theta}_{MAP} = \text{argmax}_\theta f(\theta|x)
    = \text{argmax}_\theta \frac{f(x|\theta)g(\theta)}{\int_\Theta f(x|\vartheta)g(\vartheta)d\vartheta}
    = \text{argmax}_\theta f(x|\theta)g(\theta),
\end{equation}
were the last equality is true because the normalization constant in the Bayes' formula is independent of $\theta$.

**Remark:**
Since $\log$ is a monotonic function, it is often the case that we use the logarithm of the likelihood or posterior for maximization (or negative of the logarithm for minimization).


Now suppose we have a continuous random variable $\theta \sim \mathcal{N}(\mu,\sigma^2)$. We wish to infer its mean and variance as we obtain normally distributed measurements sequentially. For the case of a random mean, $\mu$, and fixed variance, $\sigma^2$:

### C. (20 pts) 
You are responsible for purchasing a sensor that can measure the range (distance) to an object. 
Sensor I (\$100) and II (\$500) are both used to measure the range to an object. 
Suppose the measurements are noisy values of the range, $x$, such that $z \sim \mathcal{N}(\mu_z,\sigma_z^2)$ with variances of 1 (I) and 0.64 (II). The measurements obtained from these sensors can be seen in Table I and II. Parameterize the prior of $x$ with $\mu = 0$ and $\sigma ^2 = 1000$. Using the derivations from part B, write a Matlab function that takes data as input and solves the inference recursively.
C.1 Use the sensor data and the Matlab function to infer the mean and variance of the normally distributed random variable $x$ conditioned only on $z_1$.
C.2 Use the sensor data and the Matlab function to infer the mean and variance of the normally distributed random variable $x$ conditioned only on $z_2$.
C.3 Why is it that $x$ is more precise~\footnote{\href{https://en.wikipedia.org/wiki/Precision\_(statistics)}{https://en.wikipedia.org/wiki/Precision\_(statistics)}} when conditioned on $z_1$ even though sensor II is more accurate? which sensor do you recommend to be purchased?

Table 1: Sensor I Data
    
| N   | $Z_1$     |
| --- | --------- |
| 1   | 10.6715   |
| 2   | 8.7925    |
| 3   | 10.7172   |
| 4   | 11.6302   |
| 5   | 10.4889   |
| 6   | 11.0347   |
| 7   | 10.7269   |
| 8   | 9.6966    |
| 9   | 10.2939   |
| 10  | 9.2127    |

Table 2: Sensor II Data
    
| N   | $Z_2$     |
| --- | --------- |
| 1   | 10.7107   |
| 2   | 9.0823    |
| 3   | 9.1449    |
| 4   | 9.3524    |
| 5   | 10.2602   |

In [11]:
# data from sensor I
z1 = [10.6715, 8.7925, 10.7172, 11.6302, 10.4889, 11.0347, 10.7269, 9.6966, 10.2939, 9.2127]

# data from sensor II
z2 = [10.7107, 9.0823, 9.1449, 9.3524, 10.2602]

# noise variance of sensor I
sigma_z1 = 1

# noise variance of sensor II
sigma_z2 = 0.64

# non-informative prior
mu_1 = 0
sigma_1 = 1000
mu_2 = 0
sigma_2 = 1000

# MAP Bayesian inference using Gaussian prior and likelihood
def Inference(mu, sigma, sigma_z, z):
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
    var_z = 1/sigma_z
    var = 1/sigma
    cov = var_z + var
    sigma = 1/cov
    mu = (var_z*z + var*mu)*sigma


#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################
    return mu, sigma

# recursive inference with data from sensor I and sensor II
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
# run inferece using z1
for z in z1:
    mu_1, sigma_1 = Inference(mu_1, sigma_1, sigma_z1, z)

for z in z2:
    mu_2, sigma_2 = Inference(mu_2, sigma_2, sigma_z2, z)

# run inferece using z2

#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

print("sensor   mean  variance  precision(1/variance)")
print("  I%11.3f% 0.3f% 0.3f" % (mu_1, sigma_1, 1/sigma_1))
print(" II%11.3f% 0.3f% 0.3f" % (mu_2, sigma_2, 1/sigma_2))

sensor   mean  variance  precision(1/variance)
  I     10.325 0.100 10.001
 II      9.709 0.128 7.813


### D. (5 pts)
Use a Kalman filter class and write a program to solve C1 and C2. Compare your results with the MAP estimator. What is the conclusion now and why?

In [12]:
# reset non−informative prior
mu_1 = 0
sigma_1 = 1000
mu_2 = 0
sigma_2 = 1000

# Kalman Filter Measurement Update
def KF_update(mu, sigma, sigma_R, z):
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
    var_z = 1/sigma_R
    var = 1/sigma

    k = sigma/(sigma+sigma_R)
    cov = var_z + var
    sigma = 1/cov
    # mu = (var_z*z + var*mu)*sigma
    mu = mu + k * (z-mu)

    return mu, sigma

#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

# recursive inference with data from sensor I and sensor II
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
# run inferece using z1
for z in z1:
    mu_1, sigma_1 = KF_update(mu_1, sigma_1, sigma_z1, z)

for z in z2:
    mu_2, sigma_2 = KF_update(mu_2, sigma_2, sigma_z2, z)

# run inferece using z2


#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

print("sensor   mean  variance  precision(1/variance)")
print("  I%11.3f%8.3f%10.3f" % (mu_1, sigma_1, 1/sigma_1))
print(" II%11.3f%8.3f%10.3f" % (mu_2, sigma_2, 1/sigma_2))

sensor   mean  variance  precision(1/variance)
  I     10.325   0.100    10.001
 II      9.709   0.128     7.813
